# 🚀 Day 4-1: 장바구니 속 숨은 패턴 찾기, 연관규칙 분석 (Association Rule Mining) 🛒

"이 상품을 구매한 고객은 이 상품도 함께 구매했어요\!"

온라인 쇼핑몰이나 마트에서 흔히 볼 수 있는 문구죠? 

바로 이 추천의 배후에는 **연관규칙 분석(Association Rule Mining)** 이라는 강력한 데이터 분석 기법이 숨어있습니다.

연관규칙 분석은 대규모 데이터 속에서 항목들 간의 '만약 A가 등장하면, B도 함께 등장할 것이다'($A \rightarrow B$) 형태의 유용한 관계, 즉 **연관성** 을 찾아내는 비지도 학습 방법입니다. 

가장 대표적인 활용 사례가 바로 **장바구니 분석(Market Basket Analysis)** 입니다. 

고객들의 구매 기록(장바구니)을 분석하여 어떤 상품들이 함께 구매되는 경향이 있는지 파악하고, 이를 매장 진열, 상품 추천, 할인 프로모션 등 다양한 마케팅 전략에 활용할 수 있습니다.

이번 시간에는 연관규칙 분석의 핵심 원리를 이해하고, Python 라이브러리 `mlxtend`를 활용하여 실제 데이터 속에서 의미 있는 패턴을 찾아내는 방법을 배워보겠습니다.

**오늘의 핵심 포인트:**

  * **지지도 (Support)**: 얼마나 자주 함께 등장하는가?
  
  * **신뢰도 (Confidence)**: A를 샀을 때, B를 살 확률은?
  * **향상도 (Lift)**: 두 상품의 관계가 우연일까, 필연일까?

-----

<img src="images/asso_rule1.png" width="800">
<img src="images/asso_rule2.png" width="800">


### 1. 연관규칙의 핵심 3총사: 지지도, 신뢰도, 향상도

연관규칙은 단순히 "A와 B가 함께 나왔다"에서 그치지 않고, 그 관계가 얼마나 의미 있는지 측정하는 '자', 즉 평가지표를 사용합니다. 가장 중요한 세 가지 지표인 지지도, 신뢰도, 향상도에 대해 알아봅시다.

간단한 예시를 위해 5명의 고객 장바구니 데이터가 있다고 가정해 보겠습니다.

| 고객 ID | 장바구니 |
| :--- | :--- |
| 1 | `[우유, 기저귀, 빵]` |
| 2 | `[맥주, 기저귀, 과자]` |
| 3 | `[우유, 콜라]` |
| 4 | `[맥주, 기저귀]` |
| 5 | `[우유, 빵, 콜라]` |


#### 1.1 지지도 (Support)

> **"전체 거래 중에서 두 아이템이 동시에 포함된 거래의 비율은 얼마나 될까?"**

**지지도**는 전체 데이터에서 특정 아이템 집합(Itemset)이 얼마나 자주 발생하는지를 나타내는 지표입니다. 

전체 거래 횟수 중 특정 아이템 집합이 포함된 거래 횟수의 비율로 계산합니다.

$Support(A \rightarrow B) = P(A \cap B) = \frac{\text{A와 B를 모두 포함하는 거래 수}}{\text{전체 거래 수}}$

  * **의미**: 지지도는 규칙이 전체 데이터에서 얼마나 **보편적으로** 나타나는지를 보여줍니다. 지지도가 너무 낮다면 그 규칙은 일부 거래에서만 우연히 발생한 것일 수 있으므로, 일반적으로 최소 지지도(minimum support)를 설정하여 의미 없는 규칙들을 걸러냅니다.

  * **예시**: `{기저귀, 맥주}`의 지지도를 계산해 봅시다.

      * 전체 거래 수: 5
      * `기저귀`와 `맥주`를 모두 포함하는 거래: 고객 2, 4 (2건)
      * $Support(\text{{기저귀, 맥주}}) = \frac{2}{5} = 0.4$


#### 1.2 신뢰도 (Confidence)

> **"A 아이템을 구매한 고객이 B 아이템을 추가로 구매할 확률은 얼마나 될까?"**

**신뢰도**는 'A를 포함하는 거래 중에서 B도 함께 포함하는 거래'의 비율을 나타내는 조건부 확률입니다. 

즉, $A \rightarrow B$ 라는 규칙이 얼마나 신뢰할 만한지를 측정합니다.

$Confidence(A \rightarrow B) = P(B | A) = \frac{P(A \cap B)}{P(A)} = \frac{\text{A와 B를 모두 포함하는 거래 수}}{\text{A를 포함하는 거래 수}}$

  * **의미**: 신뢰도는 A와 B의 **연관성 방향**을 알려줍니다. $Confidence(A \rightarrow B)$와 $Confidence(B \rightarrow A)$는 다른 값을 가질 수 있습니다.

  * **주의점**: 신뢰도가 높다고 항상 좋은 규칙은 아닙니다. 만약 B 자체가 매우 인기 있는 상품이라면(즉, $P(B)$가 높다면), A와 상관없이 B가 구매될 확률이 원래 높기 때문에 신뢰도가 높게 나올 수 있습니다.

  * **예시**: $`기저귀` \rightarrow `맥주`$ 규칙의 신뢰도를 계산해 봅시다.

      * `기저귀`를 포함하는 거래: 고객 1, 2, 4 (3건)
      * `기저귀`와 `맥주`를 모두 포함하는 거래: 고객 2, 4 (2건)
      * $Confidence(\text{`기저귀`} \rightarrow \text{`맥주`}) = \frac{2}{3} \approx 0.67$

#### 1.3 향상도 (Lift)

> **"A와 B가 우연히 함께 팔린 것일까, 아니면 정말로 연관성이 있는 것일까?"**

**향상도**는 A의 구매가 B의 구매 확률에 얼마나 영향을 미치는지를 측정하는 지표입니다. 

A와 B가 독립적일 때($P(A \cap B) = P(A) \times P(B)$)를 기준으로, 규칙의 신뢰도가 얼마나 더 높은지를 나타냅니다.

$Lift( A \rightarrow B ) = \frac{P(B | A)}{P(B)} = \frac{Confidence(A \rightarrow B)}{Support(B)} = \frac{Support(A \cap B)}{Support(A) \times Support(B)}$

  * **해석**:

      * **Lift \> 1**: 양의 상관관계. A를 구매하면 B를 구매할 확률이 **증가**합니다. (우리가 찾고자 하는 의미 있는 규칙\!)
      * **Lift = 1**: 독립 관계. A의 구매가 B의 구매에 아무런 영향을 주지 않습니다. (우연)
      * **Lift \< 1**: 음의 상관관계. A를 구매하면 오히려 B를 구매할 확률이 **감소**합니다. (대체재 관계일 수 있음)

  * **예시**: $`기저귀` \rightarrow `맥주`$ 규칙의 향상도를 계산해 봅시다.

      * $Confidence(\text{`기저귀`} \rightarrow \text{`맥주`}) \approx 0.67$
      * `맥주`를 포함하는 거래: 고객 2, 4 (2건) $\rightarrow Support(\text{`맥주`}) = \frac{2}{5} = 0.4$
      * $Lift(\text{`기저귀`} \rightarrow \text{`맥주`}) = \frac{0.67}{0.4} \approx 1.67$
      * 향상도가 1보다 크므로, '기저귀를 사는 행위'와 '맥주를 사는 행위'는 우연 이상의 양의 관계가 있다고 해석할 수 있습니다.

-----


### 2. Python으로 연관규칙 분석하기: `mlxtend`

이제 실제 코드로 연관규칙 분석을 수행해 봅시다. `mlxtend` 라이브러리는 연관규칙 분석을 위한 Apriori 알고리즘과 관련 기능들을 매우 편리하게 제공합니다.

먼저 필요한 라이브러리를 설치합니다.


In [ ]:
!pip install mlxtend


#### 2.1 데이터 준비 및 전처리 (Transaction Encoding)

연관규칙 분석을 하려면 데이터가 특별한 형태로 준비되어야 합니다. 각 행은 하나의 거래(Transaction)를, 각 열은 하나의 상품(Item)을 나타내며, 해당 거래에 상품이 포함되었는지 여부를 `True/False` 또는 `1/0`으로 표시하는 **One-Hot Encoded** 형태여야 합니다.

`mlxtend.preprocessing`의 `TransactionEncoder`를 사용하면 파이썬 리스트 형태의 데이터를 손쉽게 변환할 수 있습니다.

**[코드] 데이터셋 생성 및 변환**

라이브러리 및 데이터셋 로드

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [3]:
path = "../datasets/ml/groceries/groceries_dataset.csv"
raw_df = pd.read_csv(path)
raw_df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [4]:
raw_df.shape

(38765, 3)

데이터 전처리

In [5]:
from dateutil import parser
raw_df.Member_number = raw_df.Member_number.astype(str)
raw_df.Date = raw_df.Date.apply(parser.parse)

EDA / 시각화

In [6]:
gantt_df = raw_df.groupby(['Member_number']).agg(
    customer_id=('Member_number', 'first'),
    start=('Date', 'min'),
    end=('Date', 'max'),
    period=('Date', lambda x: (x.max() - x.min()).days + 1)
).reset_index(drop=True).sort_values(by='period', ascending=False)
gantt_df.head()

,customer_id,start,end,period
2570,3641,2014-01-02,2015-12-27,725
159,1165,2014-01-04,2015-12-29,725
1918,2974,2014-01-01,2015-12-26,725
3274,4364,2014-01-02,2015-12-24,722
2073,3133,2014-01-04,2015-12-25,721


In [7]:
# 간트 차트 그리기
from plotly import express as px

fig = px.timeline(gantt_df.head(50), x_start='start', x_end='end', y='customer_id')
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(height=400)
fig.show()

fig2 = px.timeline(gantt_df.iloc[600:650], x_start='start', x_end='end', y='customer_id')
fig2.update_yaxes(categoryorder='total ascending')
fig2.update_layout(height=400)
fig2.show()

In [8]:
# Period의 분포
fig = px.histogram(gantt_df, x='period', nbins=200, title='이용기간 분포')
fig.show()

In [9]:
transaction_by_1d_df = raw_df.groupby(['Member_number', 'Date']).agg(
    items=('itemDescription', lambda x: list(x)),
    item_count=('itemDescription', 'size')
).reset_index().rename(columns={"Member_number":"customer_id"})
transaction_by_1d_df.head()

,customer_id,Date,items,item_count
0,1000,2014-06-24,"[whole milk, pastry, salty snack]",3
1,1000,2015-03-15,"[sausage, whole milk, semi-finished bread, yog...",4
2,1000,2015-05-27,"[soda, pickled vegetables]",2
3,1000,2015-07-24,"[canned beer, misc. beverages]",2
4,1000,2015-11-25,"[sausage, hygiene articles]",2


In [10]:
# item_count 별 빈도수 계산
agg_cnt_df = transaction_by_1d_df.item_count.value_counts().sort_index()


fig = px.bar(
    x=agg_cnt_df.index,
    y=agg_cnt_df.values,
    labels={'x': '함께 구매한 상품 수', 'y': '빈도수'},
    title='장바구니 내 상품 수 분포'
)

fig.update_layout(
    xaxis_title='상품 수',
    yaxis_title='빈도수',
    width=800,
    height=500
)

fig.update_traces(
    text=agg_cnt_df.values,
    textposition='outside'
)

fig.show()

연관규칙용 데이터셋으로 변환

In [11]:
# 샘플 장바구니 데이터 (리스트의 리스트)
dataset = transaction_by_1d_df['items'].tolist()
dataset[:10]

[['whole milk', 'pastry', 'salty snack'],
 ['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['soda', 'pickled vegetables'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['sausage', 'whole milk', 'rolls/buns'],
 ['whole milk', 'soda'],
 ['frankfurter', 'soda', 'whipped/sour cream'],
 ['frankfurter', 'curd'],
 ['beef', 'white bread']]

In [12]:
# TransactionEncoder 객체 생성 및 학습
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

# One-Hot Encoded 배열을 DataFrame으로 변환
# 열(Column) 이름은 te.columns_에 저장된 상품 이름으로 설정
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
14959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14961,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### 2.2 Apriori 알고리즘으로 빈번한 아이템셋 찾기

<u>Apriori 알고리즘</u>은 연관규칙 분석의 핵심 알고리즘입니다. 이 알고리즘의 원리는 간단합니다.

> "만약 `{맥주, 기저귀}`라는 아이템 셋이 드물게 발생한다면(지지도가 낮다면), 그것의 상위 집합인 `{맥주, 기저귀, 땅콩}`은 더 드물게 발생할 것이므로 굳이 탐색할 필요가 없다."

이처럼 <u>최소 지지도(min\_support)</u>를 기준으로, 기준을 넘지 못하는 아이템 셋을 가지치기(pruning)하여 탐색의 효율을 높이는 방식입니다.

`mlxtend.frequent_patterns`의 `apriori` 함수를 사용하여 빈번한 아이템 셋을 찾아봅시다.

**[코드] `apriori` 함수로 빈번 아이템셋 추출**

In [13]:
from mlxtend.frequent_patterns import apriori

# min_support=0.0045: 최소 지지도를 0.0045으로 설정
# use_colnames=True: 아이템 번호 대신 아이템 이름을 사용
frequent_itemsets = apriori(df, min_support=0.0045, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.021386,(UHT-milk)
1,0.008087,(baking powder)
2,0.033950,(beef)
3,0.021787,(berries)
4,0.016574,(beverages)
...,...,...
140,0.005814,"(soda, yogurt)"
141,0.008220,"(whole milk, tropical fruit)"
142,0.005213,"(tropical fruit, yogurt)"
143,0.004611,"(whipped/sour cream, whole milk)"


위 코드를 실행하면 지지도가 0.0045 이상인 아이템들의 조합(itemsets)과 각 조합의 지지도(support)가 담긴 DataFrame이 생성됩니다.


-----


**✍️ 연습문제 1: 최소 지지도 변경하기**

`apriori` 함수에서 `min_support` 값을 `0.003`로 낮춰서 실행해 보세요. 결과로 나오는 `frequent_itemsets`의 행 개수는 어떻게 변하나요? 그 이유는 무엇일까요?

In [14]:
# [문제 1] 코드를 작성해보세요
# frequent_itemsets__prob1 = apriori(?, ?, ?)
# print(frequent_itemsets_prob1)

-----

#### 2.3 연관규칙 생성 및 해석

빈번한 아이템 셋을 찾았다면, 이제 이 조합들을 가지고 $A \rightarrow B$ 형태의 **연관규칙**을 만들 차례입니다. 

`association_rules` 함수를 사용하면 이 과정을 쉽게 처리할 수 있습니다.

**[코드] `association_rules` 함수로 규칙 생성**

In [15]:
from mlxtend.frequent_patterns import association_rules

# 빈번 아이템셋을 기반으로 연관규칙 생성
# metric="confidence": 신뢰도를 기준으로 규칙을 필터링
# min_threshold=0.001: 신뢰도가 0.001 이상인 규칙만 생성
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.001)

# 결과 확인 (frozenset은 변경 불가능한 set 자료형)
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'leverage', 'conviction']]

,antecedents,consequents,support,confidence,lift,leverage,conviction
0,(beef),(whole milk),0.004678,0.137795,0.872548,-0.000683,0.976656
1,(whole milk),(beef),0.004678,0.029623,0.872548,-0.000683,0.995541
2,(other vegetables),(bottled beer),0.004678,0.038314,0.845568,-0.000854,0.992724
3,(bottled beer),(other vegetables),0.004678,0.103245,0.845568,-0.000854,0.978973
4,(bottled beer),(whole milk),0.007151,0.157817,0.999330,-0.000005,0.999874
...,...,...,...,...,...,...,...
95,(yogurt),(tropical fruit),0.005213,0.060700,0.895720,-0.000607,0.992477
96,(whipped/sour cream),(whole milk),0.004611,0.105505,0.668077,-0.002291,0.941399
97,(whole milk),(whipped/sour cream),0.004611,0.029200,0.668077,-0.002291,0.985056
98,(whole milk),(yogurt),0.011161,0.070673,0.822940,-0.002401,0.983638


결과 DataFrame에는 다음과 같은 유용한 정보들이 포함됩니다.

  * `antecedents`: 조건절 (A)
  * `consequents`: 결과절 (B)
  * `antecedent support`: A의 지지도
  * `consequent support`: B의 지지도
  * `support`: A와 B가 함께 발생할 지지도
  * `confidence`: 규칙의 신뢰도
  * `lift`: 규칙의 향상도
  * `leverage`, `conviction`: 추가적인 평가지표

-----

In [16]:
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].describe()

,support,confidence,lift
count,100.000000,100.000000,100.000000
mean,0.006586,0.079516,0.830220
std,0.002362,0.034700,0.099863
min,0.004611,0.029200,0.621522
25%,0.004946,0.047186,0.768108
50%,0.005748,0.077008,0.816942
75%,0.007151,0.100817,0.872548
max,0.014837,0.157817,1.116150


**✍️ 연습문제 2: 의미 있는 규칙 필터링하기**
 
위에서 생성된 `rules` DataFrame에서 **향상도(lift)가 0.9보다 크고, 신뢰도(confidence)가 0.1보다 큰** 규칙들만 필터링하여 출력해보세요.

In [ ]:
# [문제 2] 코드를 작성해보세요
# filtered_rules = rules[(? > 0.9) & (? > 0.1)]
# print(filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

-----

#### 2.4 결과 시각화 및 인사이트 도출

수많은 규칙들을 테이블로만 보는 것은 비효율적입니다. `Plotly Express`를 사용해 규칙들을 시각화하면 한눈에 관계를 파악하고 인사이트를 얻기 용이합니다.

**Scatter Plot**을 이용해 지지도(Support), 신뢰도(Confidence), 향상도(Lift)를 동시에 표현해 보겠습니다.

**[코드] Plotly Express로 연관규칙 시각화**

In [ ]:
import plotly.express as px

# 시각화를 위해 frozenset을 문자열로 변환
rules['antecedents_str'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents_str'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

# Scatter Plot 생성
# x축: 지지도, y축: 신뢰도, 색상: 향상도
fig = px.scatter(rules,
                 x="support",
                 y="confidence",
                 color="lift",
                 hover_data=['antecedents_str', 'consequents_str'],
                 title="Association Rules: Support vs. Confidence (Color by Lift)")
fig.show()

이 차트에서 우리는 **오른쪽 상단에 위치하면서 색이 밝은 점**들을 주목해야 합니다. 이는 지지도와 신뢰도가 모두 높고, 향상도 역시 높아 우연 이상의 강한 연관성을 갖는 규칙을 의미하기 때문입니다.

-----


**✍️ 연습문제 3: 특정 아이템이 포함된 규칙 시각화하기**

생성된 `rules`에서 `consequents` (결과절)에 'yogurt'가 포함된 규칙들만 필터링한 후, 위와 동일한 형태로 시각화를 진행해보세요. 이를 통해 'yogurt' 구매를 유도하는 상품 조합들을 집중적으로 분석할 수 있습니다.

In [ ]:
# [문제 3] 코드를 작성해보세요

# 1. 'yogurt'가 결과절에 포함된 규칙 필터링
# is_yogurt_consequent = rules['consequents'].apply(lambda x: 'yogurt' in x)
# yogurt_rules = rules[is_yogurt_consequent]

# 2. Scatter Plot 생성
# fig_yogurt = px.scatter(...)
# fig_yogurt.show()

| 지표           | 장점                                                       | 단점                                                           |
|----------------|------------------------------------------------------------|----------------------------------------------------------------|
| 지지도(Support) | 가장 기본적인 지표로, 규칙이 데이터 세트 전체에서 얼마나 자주 발생했음을 나타냄 | 지지도만으로는 아이템 간의 실제 의존성을 정확히 파악하기 어려움 |
| 신뢰도(Confidence) | 한 아이템 집합이 주어졌을 때 다른 아이템 집합이 얼마나 자주 발생했음을 보여줌 | 결과 아이템의 전체 발생 빈도를 고려하지 않아, 오해할 수 있음  |
| 향상도(Lift)     | 두 아이템 집합의 독립성 대비 상호 의존성을 직접적으로 측정함.<br/> 1보다 크면 양의 연관, 1이면 독립, 1보다 작으면 음의 연관이 있음을 나타냄 | 두 아이템의 지지도가 매우 낮은 경우에도 높은 향상도를 나타낼 수 있어, .<br/>실제 중요도를 과대평가할 가능성 있음 |
| Leverage        | 두 아이템의 독립적인 예상 지지도와 실제 지지도의 차이를 보여줌   | 값의 범위가 작아 비교적 미세한 차이를 구분하기 어려움           |
| Conviction      | 한 아이템이 결과 아이템 없이 발생할 확률에 대한 비율을 나타내며, .<br/> 아이템 간 의존도의 강도를 직관적으로 파악할 수 있음 | 다른 지표들에 비해 계산이 복잡하고 이해하기 어려움               |
| Zhang’s Metric  | 규칙의 예상 신뢰도와 실제 신뢰도의 차이를 고려하며, .<br/>방향성을 포함한 연관성의 강도를 측정함 | 다른 지표들에 비해 덜 일반적으로 사용되어 분석가들 사이에서 덜 알려짐 |

----
#### 3.1 비개인화 추천시스템으로 활용하기

연관규칙 분석을 통해 도출된 규칙들은 추천 시스템의 기반이 될 수 있습니다. 예를 들어, 특정 상품을 구매한 고객에게 연관성이 높은 다른 상품을 추천하는 방식으로 활용할 수 있습니다. 이를 통해 고객의 구매 만족도를 높이고 매출 증대를 기대할 수 있습니다.

주요 활용 방안:
- 개인화된 상품 추천
- 교차 판매(Cross-selling) 전략 수립
- 상품 진열 및 배치 최적화
- 프로모션 기획 및 타겟팅

**[코드] 연관규칙 기반 상품 추천 함수**

함수 정의

In [45]:
def recommend_products(antecedent, rules_df, metric='confidence', top_n=5):
    """
    주어진 상품(antecedent)에 대해 연관 규칙을 기반으로 추천 상품 리스트를 반환하는 함수.
    
    Parameters:
    antecedent (str): 추천의 기준이 되는 상품.
    rules_df (DataFrame): 연관 규칙이 담긴 데이터프레임.
    metric (str): 정렬 기준 
    top_n (int): 반환할 추천 상품의 최대 개수.
    
    Returns:
    list의 개별 요소 n개: 추천 상품 리스트.
    """
    # 주어진 상품에 대한 연관 규칙 필터링
    filtered_rules = rules_df[rules_df['antecedents'].apply(lambda x: antecedent in x)]
    
    # 신뢰도(confidence)가 높은 순으로 정렬
    sorted_rules = filtered_rules.sort_values(by=metric, ascending=False)
    
    # 상위 N개의 결과에서 추천 상품(consequents) 추출
    recommendations = [""] * top_n
    cal_recommendations = sorted_rules['consequents'].head(top_n).apply(lambda x: list(x)[0]).tolist()
    for i, rec in enumerate(cal_recommendations):
        recommendations[i] = rec
    
    return recommendations[0], recommendations[1], recommendations[2], recommendations[3], recommendations[4]

In [46]:
recommend_products('yogurt', rules)

('whole milk', 'other vegetables', 'rolls/buns', 'soda', 'sausage')

In [47]:
recommend_products('yogurt', rules, metric='lift')

('sausage', 'citrus fruit', 'tropical fruit', 'rolls/buns', 'whole milk')

In [48]:
recommend_products('yogurt', rules, metric='zhangs_metric')

('sausage', 'citrus fruit', 'tropical fruit', 'rolls/buns', 'whole milk')

상품리스트 데이터 준비

In [50]:
# 상품리스트
product_list = raw_df.itemDescription.unique().tolist()
len(product_list)

167

웹인터페이스

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

# Gradio 인터페이스 설정
interface = gr.Interface(
    fn=lambda product_name: recommend_products(product_name, rules),
    inputs=gr.Dropdown(choices=product_list, label="상품 선택"),
    outputs=[gr.Textbox(label="추천상품 TOP" + str(i)) for i in range(1, 6)],
    title="연관규칙 기반 추천시스템",
    description="상품을 선택하면 다른분들이 함께 구매하는 상품 5개를 추천합니다."
)

# 인터페이스 실행, 화면 높이 조정, 0.0.0.0에서 호스팅
interface.launch(height=800, server_name="0.0.0.0")

In [53]:
interface.close() # 서버 종료

Closing server running on port: 7860


-----

### 💡 더 넓은 활용: 텍스트 분석

연관규칙은 상품 구매 데이터에만 국한되지 않습니다. 예를 들어, **고객 리뷰 데이터**에서 특정 단어들이 함께 나타나는 패턴을 분석하여 제품의 장점(예: '디자인'과 '고급스러움'이 함께 등장), 불만사항(예: '배터리'와 '광탈'이 함께 등장)을 파악하는 데에도 강력하게 활용될 수 있습니다.
